In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from PIL import Image
import admm

In [2]:
DATA_PATH = '../../../OneDrive - m.titech.ac.jp/Lab/data'
n = 64
m = 128
tau = 1e-3

In [3]:
def createD(n):
    I = sp.eye(n ** 2, format='lil')

    Dx = I - sp.lil_matrix(np.roll(I.toarray(), 1, axis=1))
    Dx[n - 1::n, :] = 0
    Dy = I - sp.lil_matrix(np.roll(I.toarray(), n, axis=1))
    Dy[-n:, :] = 0
    D0 = sp.lil_matrix((n ** 2, n ** 2))

    D = sp.block_array([[Dx, D0, D0], [D0, Dx, D0], [D0, D0, Dx], [Dy, D0, D0], [D0, Dy, D0], [D0, D0, Dy]])

    return D


D = createD(n)

c:\Users\ohara\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
captured = Image.open(f"{DATA_PATH}/capture_230516/daruma.png")
captured = captured.crop((400, 460, 860, 920)).resize((n, n))
captured = np.array(captured)
g = captured.reshape(-1, 1)

In [5]:
H = np.load(f"{DATA_PATH}/systemMatrix/H_matrix_true.npy")

In [6]:
admm = admm.Admm(H, g, D, tau)

ValueError: operands could not be broadcast together with shapes (634800,1) (16384,1) 